To Do:

SVR
Gaussian Process
Polynomial Regression
NN
Random Forest for Regression

In [1]:
# Import Packages I will need
from lex_processing import * 
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# To show plots in notebook
%matplotlib inline  

from sklearn import datasets, linear_model, preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.style.use('classic')

from scipy.stats import linregress
from scipy.stats import pearsonr
from scipy import stats

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

from collections import Counter

In [5]:
# Path is where the data I want to process is.
# For Mac
pathReagan = '/Users/Joe/dropbox/Data/Edited Data/Presidents Data/ReaganSpeeches/'
pathBush = '/Users/Joe/dropbox/Data/Edited Data/Presidents Data/BushSpeeches/'
pathTrump = '/Users/Joe/dropbox/Data/Edited Data/Presidents Data/TrumpSpeeches/'
# For Linux
#pathReagan = '/home/CAMPUS/alcantaj/Dropbox/Data/Edited Data/Presidents Data/ReaganSpeeches/'
#pathBush = '/home/CAMPUS/alcantaj/Dropbox/Data/Edited Data/Presidents Data/BushSpeeches/'
#pathTrump = '/home/CAMPUS/alcantaj/Dropbox/Data/Edited Data/Presidents Data/TrumpSpeeches/'

In [6]:
dfReagan = pd.DataFrame()
dfBush = pd.DataFrame()
dfTrump = pd.DataFrame()

In [7]:
for filename in os.listdir(pathReagan):
    if filename.endswith('txt'):
        f = open(pathReagan + filename)
        raw = f.read()
        # Clear raw of punctuation and tokenize for word counts.
        wordsNoPunct = strip_punctuation(raw)
        #hesitations = wordsNoPunct.count('—')
        wordsNoPunct.replace("—", ' ')
        wordsNoPunct = word_tokenize(wordsNoPunct)
        words = word_tokenize(raw)
        # Word Counts for certain words
        c = Counter(words)
        Fillers = c['well'] + c['so'] + c['basically'] + c['actually'] + c['literally'] + c['um'] + c['ah']
        NSNouns = c['something'] + c['anything'] + c['thing']
        LIVerbs = c['be'] + c['come'] + c['do'] + c['get'] + c['give'] + c['go'] + c['have'] + c['know'] + c['look']
        + c['make'] + c['see'] + c['tell'] + c['think'] + c['want']
        
        sents = sent_tokenize(raw)
        processed = preprocess(raw)
        lex = lexical_diversity(wordsNoPunct)
        mls = meanLengthSentence(processed)
        wordDict = wordCount(processed)
        thetuple = {'Filename': filename, 'TTR': lex,
                    'WordCount':len(wordsNoPunct), 
                    'UniqueWords':len(set(wordsNoPunct)), 
                    'MLU': mls, 'Fillers': Fillers,
                   'NSNouns': NSNouns, 'LIVerbs': LIVerbs}
        finalDict = {**thetuple, **wordDict}
        dfReagan = dfReagan.append(finalDict, ignore_index = True)

In [8]:
for filename in os.listdir(pathBush):
    if filename.endswith('txt'):
        f = open(pathBush + filename)
        raw = f.read()
        # Clear raw of punctuation and tokenize for word counts.
        wordsNoPunct = strip_punctuation(raw)
        #hesitations = wordsNoPunct.count('-')
        wordsNoPunct.replace("-", ' ')
        wordsNoPunct = word_tokenize(wordsNoPunct)
        
        words = word_tokenize(raw)
        # Word Counts for certain words
        c = Counter(words)
        Fillers = c['well'] + c['so'] + c['basically'] + c['actually'] + c['literally'] + c['um'] + c['ah']
        NSNouns = c['something'] + c['anything'] + c['thing']
        LIVerbs = c['be'] + c['come'] + c['do'] + c['get'] + c['give'] + c['go'] + c['have'] + c['know'] + c['look']
        + c['make'] + c['see'] + c['tell'] + c['think'] + c['want']
        
        sents = sent_tokenize(raw)
        processed = preprocess(raw)
        lex = lexical_diversity(wordsNoPunct)
        mls = meanLengthSentence(processed)
        wordDict = wordCount(processed)
        thetuple = {'Filename': filename, 'TTR': lex,
                    'WordCount':len(wordsNoPunct), 
                    'UniqueWords':len(set(wordsNoPunct)), 
                    'MLU': mls, 'Fillers': Fillers,
                   'NSNouns': NSNouns, 'LIVerbs': LIVerbs}
        finalDict = {**thetuple, **wordDict}
        dfBush = dfBush.append(finalDict, ignore_index = True)

In [9]:
for filename in os.listdir(pathTrump):
    if filename.endswith('txt'):
        f = open(pathTrump + filename)
        raw = f.read()
        # Clear raw of punctuation and tokenize for word counts.
        wordsNoPunct = strip_punctuation(raw)
        #hesitations = wordsNoPunct.count('-')
        wordsNoPunct.replace("-", ' ')
        wordsNoPunct = word_tokenize(wordsNoPunct)
        
        words = word_tokenize(raw)
        # Word Counts for certain words
        c = Counter(words)
        Fillers = c['well'] + c['so'] + c['basically'] + c['actually'] + c['literally'] + c['um'] + c['ah']
        NSNouns = c['something'] + c['anything'] + c['thing']
        LIVerbs = c['be'] + c['come'] + c['do'] + c['get'] + c['give'] + c['go'] + c['have'] + c['know'] + c['look']
        + c['make'] + c['see'] + c['tell'] + c['think'] + c['want']
        
        sents = sent_tokenize(raw)
        processed = preprocess(raw)
        lex = lexical_diversity(wordsNoPunct)
        mls = meanLengthSentence(processed)
        wordDict = wordCount(processed)
        thetuple = {'Filename': filename, 'TTR': lex,
                    'WordCount':len(wordsNoPunct), 
                    'UniqueWords':len(set(wordsNoPunct)), 
                    'MLU': mls, 'Fillers': Fillers,
                   'NSNouns': NSNouns, 'LIVerbs': LIVerbs}
        finalDict = {**thetuple, **wordDict}
        dfTrump = dfTrump.append(finalDict, ignore_index = True)

In [10]:
# Rearranging Columns
inserted_cols = ['Filename', 'TTR','WordCount', 'UniqueWords', 'MLU', 'Fillers', 'NSNouns', 'LIVerbs']
cols = ([col for col in inserted_cols if col in dfReagan] 
        + [col for col in dfReagan if col not in inserted_cols])
dfReagan = dfReagan[cols]

In [11]:
# Rearranging Columns
inserted_cols = ['Filename', 'TTR','WordCount', 'UniqueWords', 'MLU', 'Fillers', 'NSNouns', 'LIVerbs']
cols = ([col for col in inserted_cols if col in dfBush] 
        + [col for col in dfBush if col not in inserted_cols])
dfBush = dfBush[cols]

In [12]:
# Rearranging Columns
inserted_cols = ['Filename', 'TTR','WordCount', 'UniqueWords', 'MLU', 'Fillers', 'NSNouns', 'LIVerbs']
cols = ([col for col in inserted_cols if col in dfTrump] 
        + [col for col in dfTrump if col not in inserted_cols])
dfTrump = dfTrump[cols]

In [13]:
# Fill NA's with 0s as in this dataset, NAN represent the feature NOT occuring in a particular document.
dfReagan = dfReagan.fillna(0)
dfBush = dfBush.fillna(0)
dfTrump = dfTrump.fillna(0)

In [14]:
dfReagan.to_csv('testReagan.csv')
dfBush.to_csv('testBush.csv')
dfTrump.to_csv('testTrump.csv')

In [15]:
# For Max
testpath = '/Users/Joe/Documents/PhD/Jupyter Notebooks/'
# For Linux
# testpath = '/home/CAMPUS/alcantaj/Documents/PhD/Jupyter Notebooks/'

In [16]:
a = pd.read_csv(testpath + "dates.csv")
b = pd.read_csv(testpath + "testReagan.csv")

c = pd.read_csv(testpath + "dates2.csv")
d = pd.read_csv(testpath + "testBush.csv")

e = pd.read_csv(testpath + "dates3.csv")
f = pd.read_csv(testpath + "testTrump.csv")

In [17]:
a['JDate'] = [datetime.datetime.strptime(x, '%d/%m/%Y') for x in a['Date']]
c['JDate'] = [datetime.datetime.strptime(x, '%d/%m/%Y') for x in c['Date']]
e['JDate'] = [datetime.datetime.strptime(x, '%d/%m/%Y') for x in e['Date']]

In [18]:
a['Julian'] = [get_julian_datetime(x) for x in a['JDate']]
c['Julian'] = [get_julian_datetime(x) for x in c['JDate']]
e['Julian'] = [get_julian_datetime(x) for x in e['JDate']]

In [19]:
mergedRR = a.merge(b, on='Filename')
mergedGWHB = c.merge(d, on='Filename')
mergedDJT = e.merge(f, on='Filename')

In [20]:
mergedRR = mergedRR.sort_values(by=['JDate'])
mergedGWHB = mergedGWHB.sort_values(by=['JDate'])
mergedDJT = mergedDJT.sort_values(by=['JDate'])

In [21]:
# Create new labels (Index is in order of article date)
mergedRR = mergedRR.reset_index()
mergedRR['index'] = mergedRR.index
mergedGWHB = mergedGWHB.reset_index()
mergedGWHB['index'] = mergedGWHB.index
mergedDJT = mergedDJT.reset_index()
mergedDJT['index'] = mergedDJT.index

In [22]:
# Dropping Redundant Columns
mergedRR = mergedRR.drop(['Unnamed: 0'], axis=1)
mergedGWHB = mergedGWHB.drop(['Unnamed: 0'], axis=1)
mergedDJT = mergedDJT.drop(['Unnamed: 0'], axis=1)

In [23]:
# To avoid confusion, rename merged to df
dfRR = mergedRR
dfGWHB = mergedGWHB
dfDJT = mergedDJT

In [24]:
# Add Target Variable
yRR = dfRR['index']
yGWHB = dfGWHB['index']
yDJT = dfDJT['index']

#yRR = dfRR['Julian']
#yGWHB = dfGWHB['Julian']

In [25]:
# Calculate some new aggregate columns
dfRR['Nouns'] = dfRR['NN'] + dfRR['NNS']+ dfRR['NNP'] + dfRR['NNPS']
dfRR['Nouns/100'] = dfRR['Nouns'] / 100
dfRR['NounsNormalised'] = dfRR['Nouns'] / dfRR['WordCount']
dfRR['Adjectives'] = dfRR['JJ'] + dfRR['JJR'] + dfRR['JJS']
dfRR['Adjectives/100'] = dfRR['Adjectives'] / 100
dfRR['AdjectivesNormalised'] = dfRR['Adjectives'] / dfRR['WordCount']
dfRR['Adverbs'] = dfRR['RB'] + dfRR['RBR'] + dfRR['RBS']
dfRR['Adverbs/100'] = dfRR['Adverbs'] / 100
dfRR['AdverbsNormalised'] = dfRR['Adverbs'] / dfRR['WordCount']
dfRR['Verbs'] = dfRR['VB'] + dfRR['VBD'] + dfRR['VBG'] + dfRR['VBN'] + dfRR['VBP'] + dfRR['VBZ']
dfRR['Verbs/100'] = dfRR['Verbs'] / 100
dfRR['VerbsNormalised'] = dfRR['Verbs'] / dfRR['WordCount']

In [26]:
# Calculate some new aggregate columns
dfGWHB['Nouns'] = dfGWHB['NN'] + dfGWHB['NNS']+ dfGWHB['NNP'] + dfGWHB['NNPS']
dfGWHB['Nouns/100'] = dfGWHB['Nouns'] / 100
dfGWHB['NounsNormalised'] = dfGWHB['Nouns'] / dfGWHB['WordCount']
dfGWHB['Adjectives'] = dfGWHB['JJ'] + dfGWHB['JJR'] + dfGWHB['JJS']
dfGWHB['Adjectives/100'] = dfGWHB['Adjectives'] / 100
dfGWHB['AdjectivesNormalised'] = dfGWHB['Adjectives'] / dfGWHB['WordCount']
dfGWHB['Adverbs'] = dfGWHB['RB'] + dfGWHB['RBR'] + dfGWHB['RBS']
dfGWHB['Adverbs/100'] = dfGWHB['Adverbs'] / 100
dfGWHB['AdverbsNormalised'] = dfGWHB['Adverbs'] / dfGWHB['WordCount']
dfGWHB['Verbs'] = dfGWHB['VB'] + dfGWHB['VBD'] + dfGWHB['VBG'] + dfGWHB['VBN'] + dfGWHB['VBP'] + dfGWHB['VBZ']
dfGWHB['Verbs/100'] = dfGWHB['Verbs'] / 100
dfGWHB['VerbsNormalised'] = dfGWHB['Verbs'] / dfGWHB['WordCount']

In [27]:
# Calculate some new aggregate columns
dfDJT['Nouns'] = dfDJT['NN'] + dfDJT['NNS']+ dfDJT['NNP'] + dfDJT['NNPS']
dfDJT['Nouns/100'] = dfDJT['Nouns'] / 100
dfDJT['NounsNormalised'] = dfDJT['Nouns'] / dfDJT['WordCount']
dfDJT['Adjectives'] = dfDJT['JJ'] + dfDJT['JJR'] + dfDJT['JJS']
dfDJT['Adjectives/100'] = dfDJT['Adjectives'] / 100
dfDJT['AdjectivesNormalised'] = dfDJT['Adjectives'] / dfDJT['WordCount']
dfDJT['Adverbs'] = dfDJT['RB'] + dfDJT['RBR'] + dfDJT['RBS']
dfDJT['Adverbs/100'] = dfDJT['Adverbs'] / 100
dfDJT['AdverbsNormalised'] = dfDJT['Adverbs'] / dfDJT['WordCount']
dfDJT['Verbs'] = dfDJT['VB'] + dfDJT['VBD'] + dfDJT['VBG'] + dfDJT['VBN'] + dfDJT['VBP'] + dfDJT['VBZ']
dfDJT['Verbs/100'] = dfDJT['Verbs'] / 100
dfDJT['VerbsNormalised'] = dfDJT['Verbs'] / dfDJT['WordCount']

In [28]:
# Export dataset to csv files
dfRR.to_csv('Reagan.csv')
dfGWHB.to_csv('Bush.csv')
dfDJT.to_csv('Trump.csv')

T-Tests

In [49]:
print('RR Unique Words Mean', dfRR['UniqueWords'].mean())
print('RR Unique Words SD', dfRR['UniqueWords'].std())
print('GHWB Unique Words Mean', dfGWHB['UniqueWords'].mean())
print('GWHB Unique Words SD', dfGWHB['UniqueWords'].std())
print('DJT Unique Words Mean', dfDJT['UniqueWords'].mean())
print('DJT Unique Words SD', dfDJT['UniqueWords'].std())

RR Unique Words Mean 894.1304347826087
RR Unique Words SD 85.15178831439829
GHWB Unique Words Mean 667.7651515151515
GWHB Unique Words SD 218.66893865044307
DJT Unique Words Mean 481.82758620689657
DJT Unique Words SD 221.2892530095778


In [50]:
# T-Test comparing RR Unique Words to GWHB Unique Words
t2, p2 = stats.ttest_ind(dfRR['UniqueWords'], dfGWHB['UniqueWords'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 6.8323930043748415
P Value 0.0000000001


In [51]:
# T-Test comparing RR Unique Words to DJT Unique Words
t2, p2 = stats.ttest_ind(dfRR['UniqueWords'], dfDJT['UniqueWords'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 11.40330470919139
P Value 0.0000000000


In [52]:
# T-Test comparing GWHB Unique Words to DJT Unique Words
t2, p2 = stats.ttest_ind(dfGWHB['UniqueWords'], dfDJT['UniqueWords'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 4.1374530093401
P Value 0.0000567656


In [53]:
print('RR Non Specific Nouns Mean', dfRR['NSNouns'].mean())
print('RR Non Specific Nouns SD', dfRR['NSNouns'].std())
print('GHWB Non Specific Nouns Mean', dfGWHB['NSNouns'].mean())
print('GWHB Non Specific Nouns SD', dfGWHB['NSNouns'].std())
print('DJT Non Specific Nouns Mean', dfDJT['NSNouns'].mean())
print('DJT Non Specific Nouns SD', dfDJT['NSNouns'].std())

RR Non Specific Nouns Mean 12.717391304347826
RR Non Specific Nouns SD 4.626796556669809
GHWB Non Specific Nouns Mean 6.78030303030303
GWHB Non Specific Nouns SD 4.32163807284489
DJT Non Specific Nouns Mean 7.413793103448276
DJT Non Specific Nouns SD 8.75670046054249


In [32]:
t2, p2 = stats.ttest_ind(dfRR['NSNouns'], dfGWHB['NSNouns'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 7.87791706477377
P Value 0.0000000000


In [33]:
t2, p2 = stats.ttest_ind(dfGWHB['NSNouns'], dfDJT['NSNouns'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value -0.5746875967921552
P Value 0.5663148304


In [34]:
t2, p2 = stats.ttest_ind(dfDJT['NSNouns'], dfRR['NSNouns'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value -3.4266832280839803
P Value 0.0010069016


In [54]:
print('RR Low Imageability Verbs Mean', dfRR['LIVerbs'].mean())
print('RR Low Imageability Verbs SD', dfRR['LIVerbs'].std())
print('GHWB Low Imageability Verbs Mean', dfGWHB['LIVerbs'].mean())
print('GWHB Low Imageability Verbs SD', dfGWHB['LIVerbs'].std())
print('DJT Low Imageability Verbs Mean', dfDJT['LIVerbs'].mean())
print('DJT Low Imageability Verbs SD', dfDJT['LIVerbs'].std())

RR Low Imageability Verbs Mean 124.21739130434783
RR Low Imageability Verbs SD 17.89092015952755
GHWB Low Imageability Verbs Mean 103.45454545454545
GWHB Low Imageability Verbs SD 51.867176759321026
DJT Low Imageability Verbs Mean 84.48275862068965
DJT Low Imageability Verbs SD 75.78758883016172


In [35]:
t2, p2 = stats.ttest_ind(dfRR['LIVerbs'], dfGWHB['LIVerbs'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 2.656270279028665
P Value 0.0086270563


In [36]:
t2, p2 = stats.ttest_ind(dfGWHB['LIVerbs'], dfDJT['LIVerbs'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value 1.628241215966018
P Value 0.1054536119


In [37]:
t2, p2 = stats.ttest_ind(dfDJT['LIVerbs'], dfRR['LIVerbs'])
print('T Value', t2)
print('P Value', '{0:.10f}'.format(p2))

T Value -3.4203838009668743
P Value 0.0010271838


Pearson's Correlations

In [ ]:
r,p = pearsonr([1, 2, 3, 4], [1, 5, 7, 9])
print('R Squared', r)
print('P Value', '{0:.9f}'.format(p))

In [ ]:
r,p = pearsonr(dfRR['UniqueWords'], dfRR['index'])
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
r,p = pearsonr(dfRR['Nouns/100'], dfRR['index'])
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
r,p = pearsonr(dfGWHB['Nouns/100'], dfGWHB['index'])
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
r,p = pearsonr(dfRR['WordCount'], dfRR['index'])
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
r,p = pearsonr(dfGWHB['WordCount'], dfGWHB['index'])
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
r,p = pearsonr(dfGWHB['index'], (dfGWHB['TTR']))
print('R Squared', r)
print('P Value', '{0:.10f}'.format(p))

In [ ]:
# Plot Mean Length of Utterance over time for RR
xs = dfRR['index']
ys = dfRR['MLU']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Mean Length of Utterance over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['MLU']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Mean Length of Utterance over time for DJT
xs = dfDJT['index']
ys = dfDJT['MLU']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Unique Words over time for RR
xs = dfRR['index']
ys = dfRR['UniqueWords']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Unique Words over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['UniqueWords']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Unique Words over time for DJT
xs = dfDJT['index']
ys = dfDJT['UniqueWords']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Token Type Ratio over time for RR
xs = dfRR['index']
ys = dfRR['TTR']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Token Type Ratio over time for GWHB
xs = dfGWHB['index']
ys = dfGWHB['TTR']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Token Type Ratio over time for DJT
xs = dfDJT['index']
ys = dfDJT['TTR']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Word Count over time for RR
xs = dfRR['index']
ys = dfRR['WordCount']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Word Count over time for GHWB
xs = dfGWHB['index']
ys = dfGWHB['WordCount']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Word Count over time for DJT
xs = dfDJT['index']
ys = dfDJT['WordCount']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1)
plt.plot(xs, ys, 'o')
plt.plot(xs, trend[1] + trend[0] * xs)

In [ ]:
# Plot Nouns per 100 over time for RR
xs = dfRR['index']
ys = dfRR['Nouns/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Nouns per 100 over time for GHWB
xs = dfGWHB['index']
ys = dfGWHB['Nouns/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Nouns per 100 over time for DJT
xs = dfDJT['index']
ys = dfDJT['Nouns/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Verbs per 100 over time for RR
xs = dfRR['index']
ys = dfRR['Verbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Verbs per 100 over time for GHWB
xs = dfGWHB['index']
ys = dfGWHB['Verbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Verbs per 100 over time for DJT
xs = dfDJT['index']
ys = dfDJT['Verbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adjectives per 100 over time for RR
xs = dfRR['index']
ys = dfRR['Adjectives/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adjectives per 100 over time for GHWB
xs = dfGWHB['index']
ys = dfGWHB['Adjectives/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adjectives per 100 over time for DJT
xs = dfDJT['index']
ys = dfDJT['Adjectives/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adverbs per 100 over time for RR
xs = dfRR['index']
ys = dfRR['Adverbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adverbs per 100 over time for GHWB
xs = dfGWHB['index']
ys = dfGWHB['Adverbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
# Plot Adverbs per 100 over time for DJT
xs = dfDJT['index']
ys = dfDJT['Adverbs/100']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['NounsNormalised']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['NounsNormalised']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfDJT['index']
ys = dfDJT['NounsNormalised']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['Fillers']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['Fillers']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfDJT['index']
ys = dfDJT['Fillers']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['NSNouns']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['NSNouns']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfDJT['index']
ys = dfDJT['NSNouns']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfRR['index']
ys = dfRR['LIVerbs']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfGWHB['index']
ys = dfGWHB['LIVerbs']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
xs = dfDJT['index']
ys = dfDJT['LIVerbs']
xs = np.asarray(xs)
trend = np.polyfit(xs, ys, 1) # fit a straight line
plt.plot(xs, ys,'o')
plt.plot(xs,trend[1]+trend[0]*xs)

In [ ]:
labelsRR = dfRR['Filename']
dfRR = dfRR.drop(['Filename', 'JDate', 'Julian', 'Date', 'index'], axis=1)
labelsGHWB = dfGWHB['Filename']
dfGWHB = dfGWHB.drop(['Filename', 'JDate', 'Julian', 'Date', 'index'], axis=1)
labelsDJT = dfDJT['Filename']
dfDJT = dfDJT.drop(['Filename', 'JDate', 'Julian', 'Date', 'index'], axis=1)

In [ ]:
# Scale the data
yRRscaled = preprocessing.scale(yRR)
dfRRscaled = preprocessing.scale(dfRR)

yGWHBscaled = preprocessing.scale(yGWHB)
dfGWHBscaled = preprocessing.scale(dfGWHB)

yDJTscaled = preprocessing.scale(yDJT)
dfDJTscaled = preprocessing.scale(dfDJT)

-- LINEAR REGRESSION --

In [ ]:
resultsGridRR = pd.DataFrame()
for i in range (1,1000):
    X_train, X_test, y_train, y_test = train_test_split(dfRRscaled, yRR, test_size=0.20)
    # fit a model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    slope, intercept, r_value, p_value, std_err = linregress(predictions, y_test)
    resultsTuple = {'Slope': slope, 'intercept': intercept, 'r_value': r_value,
                    'p_value':p_value, 
                    'std_err':std_err}
    resultsGridRR = resultsGridRR.append(resultsTuple, ignore_index=True)
    

In [ ]:
resultsGridGWHB = pd.DataFrame()
for i in range (1,1000):
    X_train, X_test, y_train, y_test = train_test_split(dfGWHBscaled, yGWHB, test_size=0.20)
    # fit a model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    slope, intercept, r_value, p_value, std_err = linregress(predictions, y_test)
    resultsTuple = {'Slope': slope, 'intercept': intercept, 'r_value': r_value,
                    'p_value':p_value, 
                    'std_err':std_err}
    resultsGridGWHB = resultsGridGWHB.append(resultsTuple, ignore_index=True)

In [ ]:
resultsGridDJT = pd.DataFrame()
for i in range (1,1000):
    X_train, X_test, y_train, y_test = train_test_split(dfDJTscaled, yDJT, test_size=0.20)
    # fit a model
    lm = linear_model.LinearRegression()
    model = lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    slope, intercept, r_value, p_value, std_err = linregress(predictions, y_test)
    resultsTuple = {'Slope': slope, 'intercept': intercept, 'r_value': r_value,
                    'p_value':p_value, 
                    'std_err':std_err}
    resultsGridDJT = resultsGridDJT.append(resultsTuple, ignore_index=True)

In [ ]:
resultsGridRR.describe()

In [ ]:
resultsGridGWHB.describe()

In [ ]:
resultsGridDJT.describe()

-- GAUSSIAN PROCESSES --

In [ ]:
path = '/home/CAMPUS/alcantaj/Dropbox/'
bush_df = pd.read_csv(path + 'Bush.csv')
reagan_df = pd.read_csv(path + 'Reagan.csv')

In [ ]:
minimum = reagan_df['Julian'].min() # Smallest Julian Date
maximum = reagan_df['Julian'].max() # Maximum Julian Date

In [ ]:
reagan_df['NormalisedDate'] = (reagan_df['Julian'] - minimum) / (maximum - minimum) 

In [ ]:
reagan_df['CountDays'] = reagan_df['Julian'] - minimum

In [ ]:
reagan_df['CountDaysN'] = reagan_df['CountDays'] / 365

In [ ]:
# NEED TO DROP DATA COLUMNS HERE BEFORE RUN MODEL
reagan_df = reagan_df.drop(['Unnamed: 0', 'Filename', 'index','Date', 'JDate', 'Julian',
                           'NormalisedDate', 'CountDays'], axis=1)

In [ ]:
target = reagan_df['CountDaysN']

In [ ]:
max_target = target.max()

In [ ]:
# Instantiate a Gaussian Process model
kernel = C(1.0, (1e-3, 1e3)) * RBF(185, (1e-2, 1e4))
# kernel = RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer=15)

In [ ]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(reagan_df, target, test_size=0.20)

In [ ]:
target = np.asarray(y_train)
variables = np.asarray(X_train)

In [ ]:
# Fit to data using Maximum Likelihood Estimation of the parameters
model = gp.fit(variables, target) # This looks fine according to documentation

In [ ]:
y_pred, sigma = gp.predict(np.asarray(X_test), return_std=True)

In [ ]:
# The plot
plt.figure()
xs = [x for x in range(0, 9)]
ys = [x for x in range(0, 9)]
plt.plot(xs, ys)
plt.plot(y_test, y_pred, 'o', linestyle='None')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis(xscale = 0, yscale = 0)
plt.errorbar(y_test, y_pred, yerr=sigma, elinewidth=1, fillstyle='full', linestyle ='None')

In [ ]:
sqErr = (y_test - y_pred)**2

In [ ]:
df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred, 'sigma': sigma, 'Squared Error': sqErr})

In [ ]:
df

In [ ]:
MSE = sum(sqErr) / len(sqErr)

In [ ]:
MSE

In [ ]:
bush_df = pd.read_csv(path + 'Bush.csv')
reagan_df = pd.read_csv(path + 'Reagan.csv')

In [ ]:
minimum = reagan_df['Julian'].min() # Smallest Julian Date
maximum = reagan_df['Julian'].max() # Maximum Julian Date

In [ ]:
reagan_df['NormalisedDate'] = (reagan_df['Julian'] - minimum) / (maximum - minimum) 

In [ ]:
reagan_df['CountDays'] = reagan_df['Julian'] - minimum

In [ ]:
reagan_df['CountDaysN'] = reagan_df['CountDays'] / 365

In [ ]:
# NEED TO DROP DATA COLUMNS HERE BEFORE RUN MODEL
reagan_df = reagan_df.drop(['Unnamed: 0', 'Filename', 'index','Date', 'JDate', 'Julian',
                           'NormalisedDate', 'CountDays'], axis=1)

In [ ]:
target = reagan_df['CountDaysN']

In [ ]:
max_target = target.max()

In [ ]:
# Instantiate a Gaussian Process model
kernel = C(1.0, (1e-3, 1e3)) * RBF(191, (1e-2, 1e4))
# kernel = RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer=15)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reagan_df, target, test_size=0.20)

In [ ]:
Results = pd.DataFrame()
for i in range(0, 999):
    X_train, X_test, y_train, y_test = train_test_split(reagan_df, target, test_size=0.20)
    targetVar = np.asarray(y_train)
    variables = np.asarray(X_train)
    model = gp.fit(variables, targetVar)
    y_pred, sigma = gp.predict(np.asarray(X_test), return_std=True)
    sqErr = (y_test - y_pred)**2
    MSE = sum(sqErr)/len(sqErr)
    tuple = {'MSE': MSE, 'gp.kernel':gp.kernel, 'gp.kernel_':gp.kernel_}
    Results = Results.append(tuple, ignore_index = True)

In [ ]:
Mean = Results['MSE'].mean()
SD = Results['MSE'].std()

In [ ]:
print('The Mean is', Mean)
print('The SD is', SD)

In [ ]:
Results